In [59]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from skopt.space import Real,Integer, Categorical
from skopt import BayesSearchCV
pd.set_option('max_columns',1000)

In [69]:
# Reading training data
rf_train_ovr = pd.read_csv('../training_data/train_data.csv')
rf_train_gk = pd.read_csv('../training_data/train_data_gk.csv')
rf_train_def = pd.read_csv('../training_data/train_data_def.csv')
rf_train_mid = pd.read_csv('../training_data/train_data_mid.csv')
rf_train_fwd = pd.read_csv('../training_data/train_data_fwd.csv')
rf_train_ovr.drop(['Players','club','Position'], axis = 1, inplace = True)
rf_train_gk.drop(['Players','club','Position'], axis = 1, inplace = True)
rf_train_def.drop(['Players','club','Position'], axis = 1, inplace = True)
rf_train_mid.drop(['Players','club','Position'], axis = 1, inplace = True)
rf_train_fwd.drop(['Players','club','Position'], axis = 1, inplace = True)
rf_train_ovr.describe()

,age,short_passing,dribbling,long_passing,ball_control,interceptions,positioning,vision,marking,gk_handling,gk_reflexes,Goals-16/17,A-16/17,CS-16/17,Yellow-16/17,Red-16/17,Off-16/17,Pen_SV-16/17,Pen_M-16/17,Goals_conceded-16/17,OG-16/17,Apps-16/17,Form-16/17,Goals-15/16,A-15/16,CS-15/16,Yellow-15/16,Red-15/16,Off-15/16,Pen_SV-15/16,Pen_M-15/16,Goals_conceded-15/16,OG-15/16,Apps-15/16,Form-15/16
count,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000,1435.000000
mean,26.958885,69.314286,65.090592,63.509408,69.225087,57.338676,59.091986,63.041812,52.471777,16.096167,16.383972,2.460627,1.671080,1.978397,3.312892,0.093380,4.733798,0.045296,0.068990,11.429268,0.051568,22.233449,2.363065,2.388153,1.684321,2.282230,3.399303,0.088502,4.737282,0.038328,0.055052,10.951220,0.073868,22.698258,2.421104
std,3.976445,13.939780,19.397951,14.151960,16.538790,22.404756,21.139700,14.653066,24.415962,18.655781,19.829775,4.280647,2.307097,3.523020,2.904874,0.311884,4.777352,0.277033,0.289483,17.554251,0.236466,10.331290,1.070263,4.182450,2.436358,3.972947,2.946785,0.300813,4.863116,0.254516,0.272714,16.495606,0.282164,10.327562,1.031030
min,18.000000,12.000000,6.000000,12.000000,10.000000,10.000000,4.000000,12.000000,5.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-2.000000
25%,24.000000,66.000000,60.000000,58.000000,67.000000,36.000000,49.000000,55.000000,27.000000,8.000000,8.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.000000,1.714286,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,15.000000,1.777778
50%,27.000000,73.000000,71.000000,67.000000,74.000000,68.000000,66.000000,66.000000,63.000000,11.000000,11.000000,1.000000,1.000000,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,24.000000,2.212121,1.000000,1.000000,0.000000,3.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,25.000000,2.281250
75%,30.000000,78.000000,77.000000,74.000000,79.000000,76.000000,75.000000,74.000000,74.000000,14.000000,14.000000,3.000000,2.000000,3.000000,5.000000,0.000000,7.000000,0.000000,0.000000,22.000000,0.000000,31.000000,2.941176,3.000000,2.000000,4.000000,5.000000,0.000000,7.000000,0.000000,0.000000,22.000000,0.000000,31.000000,2.932184
max,39.000000,92.000000,97.000000,93.000000,95.000000,92.000000,95.000000,92.000000,92.000000,91.000000,90.000000,37.000000,18.000000,18.000000,14.000000,2.000000,25.000000,4.000000,3.000000,82.000000,2.000000,38.000000,8.500000,40.000000,19.000000,24.000000,17.000000,2.000000,30.000000,4.000000,3.000000,67.000000,2.000000,60.000000,9.225806


In [61]:
# Reading target data
y_ovr = pd.read_csv('../targets/targets.csv')
y_gk = pd.read_csv('../targets/gk_targets.csv')
y_def = pd.read_csv('../targets/def_targets.csv')
y_mid = pd.read_csv('../targets/mid_targets.csv')
y_fwd = pd.read_csv('../targets/fwd_targets.csv')

In [62]:
# Splitting Data into train and test
x_train_ovr, x_test_ovr, y_train_ovr, y_test_ovr = train_test_split(rf_train_ovr, y_ovr, test_size = 0.2, random_state = 42)
x_train_gk, x_test_gk, y_train_gk, y_test_gk = train_test_split(rf_train_gk, y_gk, test_size = 0.2, random_state = 42)
x_train_def, x_test_def, y_train_def, y_test_def = train_test_split(rf_train_def, y_def, test_size = 0.2, random_state = 42)
x_train_mid, x_test_mid, y_train_mid, y_test_mid = train_test_split(rf_train_mid, y_mid, test_size = 0.2, random_state = 42)
x_train_fwd, x_test_fwd, y_train_fwd, y_test_fwd = train_test_split(rf_train_fwd, y_fwd, test_size = 0.2, random_state = 42)

In [63]:
# Standard scaling the train data
stand_scale_ovr = preprocessing.StandardScaler()
stand_scale_gk = preprocessing.StandardScaler()
stand_scale_def = preprocessing.StandardScaler()
stand_scale_mid = preprocessing.StandardScaler()
stand_scale_fwd = preprocessing.StandardScaler()
x_train_ovr_scaled = stand_scale_ovr.fit_transform(x_train_ovr)
x_train_gk_scaled = stand_scale_gk.fit_transform(x_train_gk)
x_train_def_scaled = stand_scale_def.fit_transform(x_train_def)
x_train_mid_scaled = stand_scale_mid.fit_transform(x_train_mid)
x_train_fwd_scaled = stand_scale_fwd.fit_transform(x_train_fwd)

In [64]:
# Standard scaling the test data
x_test_ovr_scaled = stand_scale_ovr.transform(x_test_ovr)
x_test_gk_scaled = stand_scale_gk.transform(x_test_gk)
x_test_def_scaled = stand_scale_def.transform(x_test_def)
x_test_mid_scaled = stand_scale_mid.transform(x_test_mid)
x_test_fwd_scaled = stand_scale_fwd.transform(x_test_fwd)

In [65]:
# Function to report tuning results
def report_func(opt,X,y,name):
    opt.fit(X,y)
    best_score = opt.best_score_
    best_params = opt.best_params_
    print(name)
    print(f'Best score : {best_score}')
    print(best_params)
    return best_score,[best_params]
    

In [26]:
# Using Bayesian optimizer to tune hyperparameters for overall dataset for different base estimators
xgb_ovr = XGBRegressor()
search_space = {'eta': Real(0.01,1),
                'max_depth': Integer(2,6),
                'min_child_weight': Integer(1,6),
                'subsample':Real(0.5,1),
                }
best_params_ovr = dict()
for baseestimator in ['GP','RF','ET']:
    opt = BayesSearchCV(xgb_ovr,
                        search_space,
                        scoring='r2',
                        cv=5,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseestimator},
                        random_state=43)
    key, val = report_func(opt, x_train_ovr_scaled, y_train_ovr,'BayesSearchCV_'+baseestimator+'_ovr')
    best_params_ovr[key] = val

BayesSearchCV_GP_ovr
Best score : 0.8707301571996231
OrderedDict([('eta', 0.1225284361756046), ('max_depth', 6), ('min_child_weight', 3), ('subsample', 0.5570573036073873)])
BayesSearchCV_RF_ovr
Best score : 0.8727800924065885
OrderedDict([('eta', 0.11141783853393358), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7706965625670958)])
BayesSearchCV_ET_ovr
Best score : 0.8736658914358082
OrderedDict([('eta', 0.09626201003258189), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7898083973861916)])


In [27]:
# Using Bayesian optimizer to tune hyperparameters for goalkeeper dataset for different base estimators
xgb_gk = XGBRegressor()
search_space = {'eta': Real(0.01,1),
                'max_depth': Integer(2,6),
                'min_child_weight': Integer(1,6),
                'subsample':Real(0.5,1),
                }
best_params_gk = dict()
for baseestimator in ['GP','RF','ET']:
    opt = BayesSearchCV(xgb_gk,
                        search_space,
                        scoring='r2',
                        cv=5,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseestimator},
                        random_state=43)
    key, val = report_func(opt, x_train_gk_scaled, y_train_gk,'BayesSearchCV_'+baseestimator+'_gk')
    best_params_gk[key] = val

BayesSearchCV_GP_gk
Best score : 0.8523254981027463
OrderedDict([('eta', 0.21094922811141983), ('max_depth', 4), ('min_child_weight', 4), ('subsample', 0.8330115682263901)])
BayesSearchCV_RF_gk
Best score : 0.8626267137845529
OrderedDict([('eta', 0.34705870558449786), ('max_depth', 4), ('min_child_weight', 4), ('subsample', 0.5165568722998422)])
BayesSearchCV_ET_gk
Best score : 0.8626267137845529
OrderedDict([('eta', 0.34705870558449786), ('max_depth', 4), ('min_child_weight', 4), ('subsample', 0.5165568722998422)])


In [28]:
# Using Bayesian optimizer to tune hyperparameters for defender dataset for different base estimators
xgb_def = XGBRegressor()
search_space = {'eta': Real(0.01,1),
                'max_depth': Integer(2,6),
                'min_child_weight': Integer(1,6),
                'subsample':Real(0.5,1),
                }
best_params_def = dict()
for baseestimator in ['GP','RF','ET']:
    opt = BayesSearchCV(xgb_def,
                        search_space,
                        scoring='r2',
                        cv=5,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseestimator},
                        random_state=43)
    key, val = report_func(opt, x_train_def_scaled, y_train_def,'BayesSearchCV_'+baseestimator+'_def')
    best_params_def[key] = val

BayesSearchCV_GP_def
Best score : 0.8223303453629717
OrderedDict([('eta', 0.091905865632867), ('max_depth', 6), ('min_child_weight', 3), ('subsample', 0.8429667729235724)])
BayesSearchCV_RF_def
Best score : 0.8188221984133688
OrderedDict([('eta', 0.11141783853393358), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7706965625670958)])
BayesSearchCV_ET_def
Best score : 0.8191896161182294
OrderedDict([('eta', 0.09626201003258189), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7898083973861916)])


In [29]:
# Using Bayesian optimizer to tune hyperparameters for midfielder dataset for different base estimators
xgb_mid = XGBRegressor()
search_space = {'eta': Real(0.01,1),
                'max_depth': Integer(2,6),
                'min_child_weight': Integer(1,6),
                'subsample':Real(0.5,1),
                }
best_params_mid= dict()
for baseestimator in ['GP','RF','ET']:
    opt = BayesSearchCV(xgb_mid,
                        search_space,
                        scoring='r2',
                        cv=5,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseestimator},
                        random_state=43)
    key, val = report_func(opt, x_train_mid_scaled, y_train_mid,'BayesSearchCV_'+baseestimator+'_mid')
    best_params_mid[key] = val

BayesSearchCV_GP_mid
Best score : 0.8727547893590659
OrderedDict([('eta', 0.22292644491870103), ('max_depth', 2), ('min_child_weight', 4), ('subsample', 0.79772847290864)])
BayesSearchCV_RF_mid
Best score : 0.8764975117893458
OrderedDict([('eta', 0.11141783853393358), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7706965625670958)])
BayesSearchCV_ET_mid
Best score : 0.8778316725869283
OrderedDict([('eta', 0.29884192918279395), ('max_depth', 2), ('min_child_weight', 2), ('subsample', 0.9187174769966638)])


In [30]:
# Using Bayesian optimizer to tune hyperparameters for forward dataset for different base estimators
xgb_fwd = XGBRegressor()
search_space = {'eta': Real(0.01,1),
                'max_depth': Integer(2,6),
                'min_child_weight': Integer(1,6),
                'subsample':Real(0.5,1),
                }
best_params_fwd = dict()
for baseestimator in ['GP','RF','ET']:
    opt = BayesSearchCV(xgb_fwd,
                        search_space,
                        scoring='r2',
                        cv=5,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseestimator},
                        random_state=43)
    key, val = report_func(opt, x_train_fwd_scaled, y_train_fwd,'BayesSearchCV_'+baseestimator+'_fwd')
    best_params_fwd[key] = val

BayesSearchCV_GP_fwd
Best score : 0.8595094616716026
OrderedDict([('eta', 0.1128277393545864), ('max_depth', 3), ('min_child_weight', 3), ('subsample', 0.9928580512139632)])
BayesSearchCV_RF_fwd
Best score : 0.8629093914255491
OrderedDict([('eta', 0.11141783853393358), ('max_depth', 5), ('min_child_weight', 3), ('subsample', 0.7706965625670958)])
BayesSearchCV_ET_fwd
Best score : 0.8656440759909559
OrderedDict([('eta', 0.20094858213136646), ('max_depth', 2), ('min_child_weight', 2), ('subsample', 0.8634446088768697)])


In [38]:
# Finding the parameter set of the best score for various base estimators
best_params_ovr_val = best_params_ovr[max(list(best_params_ovr.keys()))]
best_params_gk_val = best_params_gk[max(list(best_params_gk.keys()))]
best_params_def_val = best_params_def[max(list(best_params_def.keys()))]
best_params_mid_val = best_params_mid[max(list(best_params_mid.keys()))]
best_params_fwd_val = best_params_fwd[max(list(best_params_fwd.keys()))]

In [66]:
# Using the best parameters to train on the complete training dataset and position wise datasets
new_xgb_ovr = XGBRegressor(eta = best_params_ovr_val[0]['eta'],
                           max_depth = best_params_ovr_val[0]['max_depth'],
                           min_child_weight = best_params_ovr_val[0]['min_child_weight'],
                           subsample = best_params_ovr_val[0]['subsample'])
new_xgb_gk = XGBRegressor(eta = best_params_gk_val[0]['eta'],
                           max_depth = best_params_gk_val[0]['max_depth'],
                           min_child_weight = best_params_gk_val[0]['min_child_weight'],
                           subsample = best_params_gk_val[0]['subsample'])
new_xgb_def = XGBRegressor(eta = best_params_def_val[0]['eta'],
                           max_depth = best_params_def_val[0]['max_depth'],
                           min_child_weight = best_params_def_val[0]['min_child_weight'],
                           subsample = best_params_def_val[0]['subsample'])
new_xgb_mid = XGBRegressor(eta = best_params_mid_val[0]['eta'],
                           max_depth = best_params_mid_val[0]['max_depth'],
                           min_child_weight = best_params_mid_val[0]['min_child_weight'],
                           subsample = best_params_mid_val[0]['subsample'])
new_xgb_fwd = XGBRegressor(eta = best_params_fwd_val[0]['eta'],
                           max_depth = best_params_fwd_val[0]['max_depth'],
                           min_child_weight = best_params_fwd_val[0]['min_child_weight'],
                           subsample = best_params_fwd_val[0]['subsample'])

In [67]:
new_xgb_ovr = new_xgb_ovr.fit(x_train_ovr_scaled,y_train_ovr.values.ravel())
new_xgb_gk = new_xgb_gk.fit(x_train_gk_scaled,y_train_gk.values.ravel())
new_xgb_def = new_xgb_def.fit(x_train_def_scaled,y_train_def.values.ravel())
new_xgb_mid = new_xgb_mid.fit(x_train_mid_scaled,y_train_mid.values.ravel())
new_xgb_fwd = new_xgb_fwd.fit(x_train_fwd_scaled,y_train_fwd.values.ravel())

In [68]:
# Calculating variance score where 1 signifies a perfect prediction for test data
print(f"Overall score: {new_xgb_ovr.score(x_test_ovr_scaled, y_test_ovr)}")
print(f"GK score: {new_xgb_gk.score(x_test_gk_scaled, y_test_gk)}")
print(f"DEF score: {new_xgb_def.score(x_test_def_scaled, y_test_def)}")
print(f"MID score: {new_xgb_mid.score(x_test_mid_scaled, y_test_mid)}")
print(f"FWD score: {new_xgb_fwd.score(x_test_fwd_scaled, y_test_fwd)}")

Overall score: 0.8829249329095199
GK score: 0.8746952550365583
DEF score: 0.7890861974959302
MID score: 0.8386914454870552
FWD score: 0.8997437487276537
